In [21]:
import pandas as pd
import requests
import numpy as np
import requests
from bs4 import BeautifulSoup
import re



In [22]:
sal_data = pd.read_csv('sal_urls.csv')
sal_data.columns = ['index', 'url']

In [23]:
sal_data 

,index,url
0,10871,https://www.google.com/aclk?adurl=&ai=DChcSEwj...
1,10872,https://www.google.com/shopping/product/166466...
2,10873,https://www.google.com/shopping/product/777811...
3,10874,https://www.google.com/aclk?adurl=&ai=DChcSEwi...
4,10875,https://www.google.com/shopping/product/777811...
...,...,...
10866,21737,NaN
10867,21738,NaN
10868,21739,NaN
10869,21740,NaN


In [24]:
# take data from row 3917
# sal_data = sal_data.iloc[3917:]

In [25]:
sal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10871 entries, 0 to 10870
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   10871 non-null  int64 
 1   url     9133 non-null   object
dtypes: int64(1), object(1)
memory usage: 170.0+ KB


In [26]:
sal_data

,index,url
0,10871,https://www.google.com/aclk?adurl=&ai=DChcSEwj...
1,10872,https://www.google.com/shopping/product/166466...
2,10873,https://www.google.com/shopping/product/777811...
3,10874,https://www.google.com/aclk?adurl=&ai=DChcSEwi...
4,10875,https://www.google.com/shopping/product/777811...
...,...,...
10866,21737,NaN
10867,21738,NaN
10868,21739,NaN
10869,21740,NaN


In [27]:
def get_pdp(URL):
    try:
        if URL.count("google",0,35) or URL.count("yahoo",0,35) or URL.count("bing",0,35) or URL.count("youtube",0,35) or URL.count("facebook",0,35) or URL.count("wikipedia",0,35)> 0:
            class_pdp = "search_page"
        elif len(URL) < 35:
            class_pdp = "skipped"
        elif all(x in URL for x in ['amazon','/gp/aw/d/']) or \
            all(x in URL for x in ['amazon','/dp/']) or \
            all(x in URL for x in ['target','/p/']) or \
            all(x in URL for x in ['walmart','/ip/']) or \
            all(x in URL for x in ['walmart','track']):
            class_pdp = "YES"
        elif all(x in URL for x in ['amazon','/stores/']) or \
            all(x in URL for x in ['target','/b/']) or \
            all(x in URL for x in ['amazon','i=']) or \
            all(x in URL for x in ['target','/c/']) or \
            all(x in URL for x in ['walmart','browse']) or \
            all(x in URL for x in ['walmart','/cp']) or \
            all(x in URL for x in ['target','searchTerm']) or \
            all(x in URL for x in ['walmart','search']) or \
            all(x in URL for x in ['amazon','/goldbox']) or \
            all(x in URL for x in ['amazon','/offers']) or \
            all(x in URL for x in ['amazon','/angebot']) or \
            all(x in URL for x in ['target','/top-deals']) or \
            all(x in URL for x in ['target','/circle']) or \
            all(x in URL for x in ['walmart','/m']) or \
            all(x in URL for x in ['walmart','/deals']) or \
            all(x in URL for x in ['amazon','Best-Sellers']) or \
            all(x in URL for x in ['amazon','/ask','/questions']) or \
            all(x in URL for x in ['amazon','/product-reviews/']) or \
            all(x in URL for x in ['amazon','/c/ref=mw_dp_buy_crt']) or \
            all(x in URL for x in ['amazon','/cart/']) or \
            all(x in URL for x in ['amazon','/huc/view.html?']) or \
            all(x in URL for x in ['target','/co-cart']) or \
            all(x in URL for x in ['target','/cart']) or \
            all(x in URL for x in ['amazon','bestsellers','Best-Sellers']):
            class_pdp = "Other_pages"
        else:
            HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
            webpage = requests.get(URL, headers=HEADERS, timeout=25)
            soup = BeautifulSoup(webpage.content, "html5lib")
            text = soup.find_all(text=True)
            if text == None:
                class_pdp = "No Data"
            else:
                new = []
                for i in text:
                    new.append(i.lower())
                    mystr =" ".join(new)
                pdp = [i for i in new if i == "add to cart" or i == " add to cart " or i == "add to shipping" or 
                i == " add to shipping " or i == "add to basket" or i == " add to basket " or i == "buy now" or 
                i == " buy now " or i == "add item" or i == " add item " or i == "in the cart" or i == " in the cart " or 
                i == "description" or i == "item description" or i == "product description" or i == "about this product" or 
                i == "about this item" or i == "overview"  or i == "over view" or i == " add for shipping " or i == "add for shipping" or i == "in den warenkorb"]
                if len(pdp) > 0 and len(pdp)< 10:
                    class_pdp = "YES"
                elif mystr.count("add to basket" or "add to shipping" or "add to cart" or "buy now" or "add item" or "in the cart" or " add to cart " or " add to shipping " or " add to basket " or " buy now " or " add item " or " in the cart " or "add for shipping" or " add for shipping " or "in den warenkorb")> 0 \
                    & mystr.count("add to basket" or "add to shipping" or "add to cart" or "buy now" or "add item" or "in the cart" or " add to cart " or " add to shipping " or " add to basket " or " buy now " or " add item " or " in the cart " or "add for shipping" or " add for shipping " or "in den warenkorb")< 15:
                    class_pdp = "YES"
                else:
                    class_pdp = "NO"
    except Exception as e:
        print(e)
        class_pdp = "error or timeout"
    return class_pdp




In [ ]:
def get_pdp(URL):
    try:
        if URL.count("google",0,35) or URL.count("yahoo",0,35) or URL.count("bing",0,35) or URL.count("youtube",0,35) or URL.count("facebook",0,35) or URL.count("wikipedia",0,35)> 0:
            class_pdp = "search_page"
        elif len(URL) < 35:
            class_pdp = "skipped"
        elif all(x in URL for x in ['amazon','/gp/aw/d/']) or \
            all(x in URL for x in ['amazon','/dp/']) or \
            all(x in URL for x in ['target','/p/']) or \
            all(x in URL for x in ['walmart','/ip/']) or \
            all(x in URL for x in ['walmart','track']):
            class_pdp = "YES"
        elif all(x in URL for x in ['amazon','/stores/']) or \
            all(x in URL for x in ['target','/b/']) or \
            all(x in URL for x in ['amazon','i=']) or \
            all(x in URL for x in ['target','/c/']) or \
            all(x in URL for x in ['walmart','browse']) or \
            all(x in URL for x in ['walmart','/cp']) or \
            all(x in URL for x in ['target','searchTerm']) or \
            all(x in URL for x in ['walmart','search']) or \
            all(x in URL for x in ['amazon','/goldbox']) or \
            all(x in URL for x in ['amazon','/offers']) or \
            all(x in URL for x in ['amazon','/angebot']) or \
            all(x in URL for x in ['target','/top-deals']) or \
            all(x in URL for x in ['target','/circle']) or \
            all(x in URL for x in ['walmart','/m']) or \
            all(x in URL for x in ['walmart','/deals']) or \
            all(x in URL for x in ['amazon','Best-Sellers']) or \
            all(x in URL for x in ['amazon','/ask','/questions']) or \
            all(x in URL for x in ['amazon','/product-reviews/']) or \
            all(x in URL for x in ['amazon','/c/ref=mw_dp_buy_crt']) or \
            all(x in URL for x in ['amazon','/cart/']) or \
            all(x in URL for x in ['amazon','/huc/view.html?']) or \
            all(x in URL for x in ['target','/co-cart']) or \
            all(x in URL for x in ['target','/cart']) or \
            all(x in URL for x in ['amazon','bestsellers','Best-Sellers']):
            class_pdp = "Other_pages"
        else:
            HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
            webpage = requests.get(URL, headers=HEADERS, timeout=25)
            soup = BeautifulSoup(webpage.content, "html5lib")
            text = soup.find_all(text=True)
            if text == None:
                class_pdp = "No Data"
            else:
                new = []
                for i in text:
                    new.append(i.lower())
                    mystr =" ".join(new)
                #read items from class_pdp.txt in to a list
                with open('class_pdp.txt', 'r') as f:
                    pdp = f.read().splitlines()
                #check if any of the items in the list is in the text
                


In [28]:
for i in sal_data.url:
    class_pdp = get_pdp(i)
    f = open('class_pdp.txt', 'a+')
    f.write(class_pdp+"\n")
    f.close()

KeyboardInterrupt: 

In [10]:
f2 = open("class_pdp.txt", "r")
pdp_tag = f2.readlines()

pdp=[]
for i in pdp_tag:
    pdp.append(i[:-1])

In [11]:
pdp_df = pd.DataFrame(pdp, columns=['pdp'])
# add and index column starting from 10871
pdp_df['index'] = sal_data['index']
len_pfp = len(pdp_df)

In [12]:
len_pfp

4184

In [13]:
pdp_df

,pdp,index
0,search_page,10871
1,search_page,10872
2,search_page,10873
3,search_page,10874
4,search_page,10875
...,...,...
4179,NO,15050
4180,NO,15051
4181,search_page,15052
4182,search_page,15053


In [14]:
merged_df = pd.merge(sal_data, pdp_df, on='index')

In [15]:
merged_df

,index,url,pdp
0,10871,https://www.google.com/aclk?adurl=&ai=DChcSEwj...,search_page
1,10872,https://www.google.com/shopping/product/166466...,search_page
2,10873,https://www.google.com/shopping/product/777811...,search_page
3,10874,https://www.google.com/aclk?adurl=&ai=DChcSEwi...,search_page
4,10875,https://www.google.com/shopping/product/777811...,search_page
...,...,...,...
4179,15050,https://biccamera.rakuten.co.jp/item/871010384...,NO
4180,15051,https://review.rakuten.co.jp/item/1/269553_125...,NO
4181,15052,https://www.google.com/search?aqs=chrome..69i5...,search_page
4182,15053,https://www.youtube.com/watch?v=ZGD6t2929-k,search_page


In [16]:
df = pd.read_csv('ClickStream+PurchasesCombined_FINAL_15Feb (1).csv')
# Creat a new column for index
df['index'] = df.index

c:\VS_Code\O360\O360\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (21,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
df.columns

Index(['Name', 'Total time spent on user test', 'Average time taken on tasks',
       'Average page views per task', 'User agent', 'Custom ID',
       'Date completed user test', 'Item', 'Task/Question', 'Page views',
       'Path Analysis', 'Navigation', 'Time spent', 'Task result', 'domain',
       'UUID', 'surveyID', 'TouchPoint', 'Sequence', 'Reverse Sequence',
       'Sponsored', 'Sponsored link', 'Domain_Type', 'DTC_Page',
       'Domain_Type2', 'SearchType', 'SearchType2', 'Gender', 'Country',
       'Survey', 'FullName', 'JourneyStage', 'ProdName', 'ratings', 'OID',
       'resid', 'User Id', 'searchTerm', 'PDP Flag', 'Prev_PDP', 'Prev_Brand',
       'Brand', 'URL_Cat', 'index'],
      dtype='object')

In [18]:
df = df[['Path Analysis','index','UUID',  'PDP Flag', 'URL_Cat']]

In [19]:
df_ckeck = pd.merge(df, merged_df, on='index')

In [20]:
df_ckeck.to_csv('df_ckeck.csv')